In [42]:
from clickhouse_driver import Client
import pandas as pd 
import datetime
from datetime import datetime as dt, date

In [43]:
client = Client(host = 'localhost',
                client_name = 'python-driver',
                port = 49000,
                user='primedata',
                password='primedata',
                secure=False,
                verify=False,
                database = 'eventify_stag',
               )

In [44]:
def query_erp(CH_schema):#ch_client,schema,start_time, end_time
    query_transaction = f""" 
                select a.*,b.gender,c.Province,d.card_tier_name,e.dob from
                (
                select at,anonymous_id,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.account_id')] as account_id,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ngay_ct')] as ngay_ct,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.gio_gh')] as gio_gh,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.stt_rec')] as stt_rec,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.dich_vu')] as dich_vu,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.dvt')] as dvt,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.nguon_dh')] as nguon_dh,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_vt')] as ma_vt,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_vt2')] as ma_vt2,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_ct_km')] as ma_ct_km,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ten_ct_km')] as ten_ct_km,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_voucher')] as ma_voucher,
                    `str_properties.vals`[indexOf(`str_properties.keys`,'properties.ma_dvcs')] as ma_dvcs,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.gia_ban')] as gia_ban,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.tien_kh_nop')] as tien_kh_nop,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.t_tien_sau_ck')] as t_tien_sau_ck,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.tien_ck_voucher')] as tien_ck_voucher,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.tien_ck_voucher_tm')] as tien_ck_voucher_tm,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.tien_voucher')] as tien_voucher,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.ck')] as ck,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.pt_ck')] as pt_ck,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.pt_ck_dh')] as pt_ck_dh,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.so_luong')] as so_luong,
                    `num_properties.vals`[indexOf(`num_properties.keys`,'properties.phi_vc')] as phi_vc 
                    from {CH_schema}.events 
                    where event_name ='erp_item_order_completed' 
                    and ma_dvcs ='0102') a

                    left outer join 

                    (select anonymous_id, str_val as gender 
                    from {CH_schema}.profile_str_final_v 
                    where str_key='gender') b
                    on a.anonymous_id = b.anonymous_id

                    left outer join 

                    (select anonymous_id,str_val as Province
                    from {CH_schema}.profile_str_final_v 
                    where str_key='city') c
                    on a.anonymous_id = c.anonymous_id

                    left outer join
                    (select anonymous_id,str_val as card_tier_name 
                    from {CH_schema}.profile_str_final_v 
                    where str_key='card_tier_name') d
                    on a.anonymous_id = d.anonymous_id 

                    left outer join
                    (select anonymous_id,str_val as dob 
                    from {CH_schema}.profile_str_final_v 
                    where str_key = 'dob') e 
                    on a.anonymous_id = e.anonymous_id
            """
    iterator = client.execute_iter(query_transaction, {'max_block_size': 5000}, with_column_types=True)
    columns = next(iterator)
    transaction_data = []
    for item in iterator:
        transaction_data.append(item)
    transaction_df = pd.DataFrame(transaction_data[1:], columns=[col[0][2:] for col in columns])
    return transaction_df

In [45]:
def interval_age(age):
    if age in range(10, 20):
        return '<20'
    if age in range(20, 26):
        return '20-25'
    if age in range(26, 31):
        return '26-30'
    if age in range(31, 36):
        return '31-35'
    if age in range(36, 41):
        return '36-40'
    if age in range(41, 46):
        return '41-46'
    if age in range(46, 51):
        return '46-50'
    if age in range(51, 100):
        return ">50"
    return 'undefined'

In [46]:
def calculate_age(born):
    if born !="":
        today = date.today()
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [47]:
def convert_datetime(datetime_ob):
    if not datetime_ob:
        return datetime_ob
    try:
        return dt.strptime(datetime_ob, "%Y-%m-%dT%H:%M:%S")
    except ValueError:
        return dt.strptime(datetime_ob, "%Y-%m-%d")

In [48]:
def agg_interval_age(dataframe):
    dataframe['age'] = dataframe['dob'].apply(convert_datetime).apply(calculate_age)
    dataframe['interval_age'] = dataframe['age'].apply(interval_age)
    return dataframe

In [49]:
def replace_EmptyNanNull(x):
    if x == None or x == '' or x==np.NaN:
        return 'undefined'
    else:
        return x

In [50]:
def convert_time(datetime_ob):
    if not datetime_ob:
        return datetime_ob
    return dt.strptime(datetime_ob,"%H:%M:%S").time()

In [51]:
def interval_time(time):
        if time in list(pd.date_range("08:30:00", "13:00:00", freq="1S").time):
            return '8h30 - 13h'
        if time in list(pd.date_range("13:00:00", "18:00:00", freq="1S").time):
            return '13h - 18h'
        if time in list(pd.date_range("18:00:00", "22:00:00", freq="1S").time):
            return '18h - 22h'
        else:
            return '22h - 8h30'

In [52]:
def agg_interval_time(dataframe):
    dataframe['interval_time']= dataframe['gio_gh'].apply(convert_time).apply(interval_time)
    return dataframe

In [53]:
def detail_voucher(ma_voucher):
        if ma_voucher[0:5] =='CK214':
            return 'Voucher 300k AD Bill 1200k triệu - CT Khai trương SR Long Xuyên - HSD: 30/01/2022'
        if ma_voucher[0:5] =='CK215':
            return 'Voucher 500k AD Bill 1500k triệu - CT Khai trương SR Long Xuyên - HSD: 30/01/2022'
        if ma_voucher[0:5] =='CK216':
            return 'Voucher CK 650k AD bill từ 1 triệu - CT Khai trương SR Long Xuyên - HSD: 30/01/2022'
        return 'Không voucher'

In [54]:
def erp_pipeline(data):#ch_client,schema,start_time, end_time)\n",
    erp_result = data.copy()
    erp_result = agg_interval_age(erp_result)
    erp_result = agg_interval_time(erp_result)
    erp_result['Province'] = erp_result['Province'].apply(replace_EmptyNanNull)
    erp_result['card_tier_name'] = erp_result['card_tier_name'].apply(replace_EmptyNanNull)
    erp_result['interval_age']= erp_result['interval_age'].apply(replace_EmptyNanNull)
    erp_result['gender']=erp_result['gender'].apply(replace_EmptyNanNull)
    erp_result.drop(columns = ['age','dob','anonymous_id'],inplace=True)
    erp_result.rename(columns ={'account_id':'anonymous_id'},inplace=True)
    return erp_result

In [55]:
raw = query_erp('eventify_stag')

In [56]:
raw

,at,anonymous_id,account_id,ngay_ct,gio_gh,stt_rec,dich_vu,dvt,nguon_dh,ma_vt,...,tien_voucher,ck,pt_ck,pt_ck_dh,so_luong,phi_vc,gender,Province,card_tier_name,dob
0,2022-01-02 10:39:35,239WPbjnG0gxcyr9V2j0DNUZV2j,710895,2022-01-02T00:00:00,10:39:35,K1000000004770873BL2,,CHIEC,,AJN00701WCO03DN01395,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,,1994-02-02T00:00:00
1,2022-01-02 21:38:48,239WPlhYytM5U8Go7hcs0YGxLUZ,711075,2022-01-02T00:00:00,21:38:48,K1000000004771485BL2,,CHIEC,,ASS177S1WMO02M177093,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,MEMBER,2005-07-18T00:00:00
2,2022-01-02 21:38:48,239WPlhYytM5U8Go7hcs0YGxLUZ,711075,2022-01-02T00:00:00,21:38:48,K1000000004771485BL2,,CHIEC,,ASS258S1WCO10M258064,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,MEMBER,2005-07-18T00:00:00
3,2022-01-02 21:38:48,239WPlhYytM5U8Go7hcs0YGxLUZ,711075,2022-01-02T00:00:00,21:38:48,K1000000004771485BL2,,CHIEC,,AJN00409WCO02DN01393,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,MEMBER,2005-07-18T00:00:00
4,2022-01-02 21:38:48,239WPlhYytM5U8Go7hcs0YGxLUZ,711075,2022-01-02T00:00:00,21:38:48,K1000000004771485BL2,,CHIEC,,ATS009S1KCO03W006002,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,MEMBER,2005-07-18T00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,2022-01-01 16:51:58,236gt0kz5eQW2PMcJw2fEvqWQEC,710789,2022-01-01T00:00:00,16:51:58,K1000000004770137BL2,,CHIEC,,ABF06309KNY000000003,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,,2022-01-01T00:00:00
485,2022-01-01 16:51:58,236gt0kz5eQW2PMcJw2fEvqWQEC,710789,2022-01-01T00:00:00,16:51:58,K1000000004770137BL2,,CHIEC,,AJN00701WCO03DN01393,...,0.0,0.0,0.0,0.0,2.0,0.0,Nam,An Giang,,2022-01-01T00:00:00
486,2022-01-01 17:06:58,236hCp63A0p9YYRejQ0fwAvSgdj,710801,2022-01-01T00:00:00,17:06:58,K1000000004770225BL2,,CHIEC,,ALS13509WMO02J135104,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,,1995-11-07T00:00:00
487,2022-01-01 20:27:00,236hCsOMYxL65Jva4PwaNndswjF,710854,2022-01-01T00:00:00,20:27:00,K1000000004770509BL2,,CHIEC,,ABX06409KNY000000005,...,0.0,0.0,0.0,0.0,1.0,0.0,Nam,An Giang,,1993-10-17T00:00:00


In [57]:
raw_new = erp_pipeline(raw)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_10088/1737037030.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  dataframe['age'] = dataframe['dob'].apply(convert_datetime).apply(calculate_age)


In [19]:
raw_new['interval_age'].value_counts()

26-30        102
undefined     95
31-35         85
36-40         52
20-25         49
41-46         47
>50           14
<20            9
46-50          9
Name: interval_age, dtype: int64

# voucher


In [58]:
voucher_fact = pd.DataFrame({'ma_voucher':list(raw_new['ma_voucher'].unique())})

In [59]:
voucher_fact['voucher_name'] = voucher_fact['ma_voucher'].apply(detail_voucher)

In [60]:
voucher_fact.to_csv('voucher_fact.txt',mode = 'w', sep = ';', header = True, index = None,encoding = 'utf-8')

In [61]:
raw_new.to_csv('Long_xuyen_raw.txt',mode = 'w', sep = '\t', header = True, index = False,encoding = 'utf-8')

# test age

In [41]:
under20 = raw_new[raw_new['age']<20].copy()

In [55]:
under20.groupby('age')['account_id'].nunique()

age
0.0     11
16.0     2
18.0     1
19.0     1
Name: account_id, dtype: int64

In [61]:
raw_new.groupby('interval_age')['account_id'].nunique()

interval_age
20-25         4
26-30        19
31-35         9
36-40         8
41-46         7
<20           4
>50           4
undefined    29
Name: account_id, dtype: int64

In [66]:
raw_new.groupby('card_tier_name')['anonymous_id'].nunique()

card_tier_name
MEMBER        3
SILVER        1
undefined    80
Name: anonymous_id, dtype: int64

In [70]:
len(raw_new['anonymous_id'].unique())

81

In [71]:
test = raw_new[['anonymous_id','card_tier_name']].copy()

In [80]:
test= test.groupby('anonymous_id').agg({'card_tier_name':['unique']}).reset_index()

In [82]:
test.columns = ['anonymous_id','card_tier_name']

In [84]:
test['count'] = [len(i) for i in test['card_tier_name']]

In [90]:
test.sort_values('count',ascending = False)

,anonymous_id,card_tier_name,count
63,713719,"[undefined, MEMBER]",2
4,710854,"[undefined, MEMBER]",2
55,713473,"[undefined, MEMBER]",2
0,415708,[SILVER],1
51,713371,[undefined],1
...,...,...,...
26,713114,[undefined],1
25,713111,[undefined],1
24,713109,[undefined],1
23,713107,[undefined],1


In [88]:
test

,anonymous_id,card_tier_name,count
0,415708,[SILVER],1
1,653997,[undefined],1
2,710789,[undefined],1
3,710801,[undefined],1
4,710854,"[undefined, MEMBER]",2
...,...,...,...
76,713917,[undefined],1
77,714076,[undefined],1
78,714087,[undefined],1
79,714094,[undefined],1


In [91]:
import subprocess

In [95]:
test = subprocess.run(['python.exe',"db-fwd.sh"])

In [96]:
te

CompletedProcess(args=['python.exe', 'db-fwd.sh'], returncode=1)

In [34]:
 a= {
  "erp": {
    "at": "2021-01-01 22:35:00"
  },
  "dim_segment": {
    "at": "2022-01-18 04:13:18",
    "segment_id": "23r57Q2ivubcXyiylyUYnCwegEp"
  },
  "fact_segment": {
    "at": "2021-06-04 07:00:00",
    "segment_id": "1tPwihd7vkzjf5sCyEIL6IFLBqn",
    "user_id": "1udlaZSrUaUGjaLdO57ndaLzccy"
  },
  "user": {
    "user_id": "1uceymgCXyQ0dH1zZgqljwPGXwO"
  },
  "product": {
    "ma_vt": "11901320000.0"
  }
}

In [35]:
 b= {
  "erp": {
    "at": "2021-01-01 22:35:00"
  },
  "dim_segment": {
    "at": "2022-01-18 04:13:18",
    "segment_id": "23r57Q2ivubcXyiylyUYnCwegEp"
  },
  "fact_segment": {
    "at": "2021-06-04 07:00:00",
    "segment_id": "1tPwihd7vkzjf5sCyEIL6IFLBqn",
    "user_id": "1udlaZSrUaUGjaLdO57ndaLzccy"
  },
  "user": {
    "user_id": "1uceymgCXyQ0dH1zZgqljwPGXwO"
  },
  "product": {
    "ma_vt": "11901320000.0"
  }
}

In [32]:
 c= {
  "erp": {
    "at": "2021-01-01 22:35:00"
  },
  "dim_segment": {
    "at": "2022-01-18 04:13:18",
    "segment_id": "23r57Q2ivubcXyiylyUYnCwegEp"
  },
  "fact_segment": {
    "at": "2021-06-04 07:00:00",
    "segment_id": "1tPwihd7vkzjf5sCyEIL6IFLBqn",
    "user_id": "1udlaZSrUaUGjaLdO57ndaLzccy"
  },
  "user": {
    "user_id": "1uceymgCXyQ0dH1zZgqljwPGXw"
  },
  "product": {
    "ma_vt": "11901320000.0"
  }
}

In [36]:
a == b

True

In [33]:
a==c

False

In [37]:
from loguru import logger

In [38]:
logger()

In [41]:
client.execute('select distinct str_key from profile_str_final_v')

[('channels_reachable.onsite.notification_token',),
 ('channels_reachable.onsite.status',),
 ('firstVisit',),
 ('lastVisit',),
 ('previousVisit',),
 ('systems.goals.1uh2kNXybUQtcErIjidZ01lauWL_0_0StartReached',),
 ('systems.goals.1uh2kNXybUQtcErIjidZ01lauWL_0_0TargetReached',),
 ('systems.goals.1vO758S2AIdi0kfqaGWawBBJmAc_0_0StartReached',),
 ('systems.goals.1vO758S2AIdi0kfqaGWawBBJmAc_0_0TargetReached',),
 ('systems.goals.1vO758S2AIdi0kfqaGWawBBJmAc_0_1StartReached',),
 ('systems.goals.1veUU3DoR0cS7AYEJkIAt1zuJ9n_0_0StartReached',),
 ('systems.goals.1veUU3DoR0cS7AYEJkIAt1zuJ9n_0_0TargetReached',),
 ('systems.goals.1veUU3DoR0cS7AYEJkIAt1zuJ9n_0_1StartReached',),
 ('systems.goals.1veUU3DoR0cS7AYEJkIAt1zuJ9n_0_1TargetReached',),
 ('systems.goals.1vyJXgpS0l5pBdAXqndPYgnkmYC_0_0StartReached',),
 ('systems.goals.1vyJXgpS0l5pBdAXqndPYgnkmYC_0_0TargetReached',),
 ('systems.goals.1vyJXgpS0l5pBdAXqndPYgnkmYC_0_1StartReached',),
 ('systems.goals.1vyJXgpS0l5pBdAXqndPYgnkmYC_0_1TargetReached',),
 

In [67]:
def add_months(sourcedate, months):
    """

    :param sourcedate:
    :param months:
    :return:
    """
    if type(sourcedate) == datetime:
        sourcedate = sourcedate
    else:
        sourcedate = datetime.strptime(sourcedate,"%Y-%m-%d %H:%M:%S")
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    hour = sourcedate.hour
    minute = sourcedate.minute
    second = sourcedate.second
    return datetime(year, month, day,hour,minute,second)

In [107]:
from datetime import datetime, timedelta
import calendar

In [86]:
import time

In [101]:
a =datetime.strptime("2021-01-01 00:00:00","%Y-%m-%d %H:%M:%S")

In [84]:
add_months(datetime.strptime(a,"%Y-%m-%d %H:%M:%S"),1)

datetime.datetime(2021, 2, 1, 0, 0)

In [108]:
a+ datetime.timedelta(days=30)

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'